<a href="https://colab.research.google.com/github/bhargavi369/IoT-Data-with-Spark-Sql/blob/main/iotdata_sparksql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installing findspark
%pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Installing pyspark
%pip install pyspark
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=1c8da7181e30029805cfe128e2d0d7153414655fd4239eb1e3c6b21d5a0ca917
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
# Installing java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
     

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark.sql.functions as pyf

In [ ]:
# Creating a spark session
from pyspark.sql import DataFrame, SparkSession
spark = SparkSession.builder.appName("Analysing IoT Data").getOrCreate()
spark
     

In [ ]:
# Specifying the file path
file_location = "/content/drive/MyDrive/Colab Notebooks/iot_devices.json"
     

In [ ]:
# Reading the dataset to a dataframe
df = spark.read.json(file_location)

In [ ]:
# Displaying the number of rows and columns
print((df.distinct().count(), len(df.columns)))
     

(198164, 15)


In [ ]:
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [ ]:
df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [ ]:
# Creating a temporary view called iot
df.createOrReplaceTempView("iot")

In [ ]:
print(df)

DataFrame[battery_level: bigint, c02_level: bigint, cca2: string, cca3: string, cn: string, device_id: bigint, device_name: string, humidity: bigint, ip: string, latitude: double, lcd: string, longitude: double, scale: string, temp: bigint, timestamp: bigint]


In [ ]:
# Count how many devices are there from each country and display the output.
spark.sql("SELECT cn, COUNT(DISTINCT device_id) as Number_of_devices FROM iot GROUP BY cn ORDER BY Number_of_devices DESC").show()

+-----------------+-----------------+
|               cn|Number_of_devices|
+-----------------+-----------------+
|    United States|            68545|
|            China|            14455|
|            Japan|            12100|
|Republic of Korea|            11879|
|          Germany|             7942|
|   United Kingdom|             6486|
|           Canada|             6041|
|           Russia|             5989|
|           France|             5305|
|           Brazil|             3224|
|        Australia|             3119|
|            Italy|             2915|
|           Sweden|             2880|
|           Poland|             2744|
|      Netherlands|             2488|
|            Spain|             2310|
|           Taiwan|             2128|
|            India|             1867|
|                 |             1810|
|   Czech Republic|             1507|
+-----------------+-----------------+
only showing top 20 rows



In [ ]:
# All the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.
spark.sql("SELECT cn FROM iot WHERE c02_level > 1400 ORDER BY c02_level DESC").show()
     

+-----------------+
|               cn|
+-----------------+
|         Bulgaria|
|    United States|
|            Egypt|
|           Taiwan|
|          Germany|
|          Germany|
|           Canada|
|            Japan|
|          Germany|
|         Bulgaria|
|        Argentina|
|   United Kingdom|
|    United States|
|    United States|
|    United States|
|            China|
|Republic of Korea|
|    United States|
|           Norway|
|           Poland|
+-----------------+
only showing top 20 rows



In [ ]:
# Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids
spark.sql("SELECT device_name,device_id,cca3 FROM iot WHERE lcd = 'red' GROUP BY device_name,device_id,cca3 ORDER BY device_id").show()
     

+--------------------+---------+----+
|         device_name|device_id|cca3|
+--------------------+---------+----+
|   sensor-pad-2n2Pea|        2| NOR|
| device-mac-36TWSKiT|        3| ITA|
|sensor-pad-8xUD6p...|        8| JPN|
|sensor-pad-10Bsyw...|       10| USA|
|meter-gauge-11dlM...|       11| ITA|
|sensor-pad-16aXmI...|       16| USA|
|meter-gauge-17zb8...|       17| USA|
|meter-gauge-19eg1...|       19| USA|
|  sensor-pad-22oWV2D|       22| JPN|
|sensor-pad-24Pytz...|       24| CAN|
|  device-mac-27P5wf2|       27| KOR|
|sensor-pad-28Tsud...|       28| KOR|
|meter-gauge-47WsF9s8|       47| UKR|
|meter-gauge-534fD...|       53| SWE|
|sensor-pad-5410CW...|       54| USA|
|device-mac-57tRB0...|       57| USA|
|  sensor-pad-64djcIn|       64| USA|
|sensor-pad-663dzo...|       66| CZE|
|meter-gauge-77IKW...|       77| IND|
|sensor-pad-78PkeI...|       78| KOR|
+--------------------+---------+----+
only showing top 20 rows



In [ ]:
# All devices in countries whose batteries need replacements.
spark.sql("SELECT device_name as Device_Battery_Dead, device_id FROM iot WHERE battery_level = 0 GROUP BY Device_Battery_Dead, device_id").show()

+--------------------+---------+
| Device_Battery_Dead|device_id|
+--------------------+---------+
|meter-gauge-1075K...|      107|
|sensor-pad-354ztT...|      354|
|device-mac-2757kb...|     2757|
|meter-gauge-41092...|     4109|
|therm-stick-5485e...|     5485|
|sensor-pad-9038r8...|     9038|
|sensor-pad-11448P...|    11448|
|sensor-pad-12444M...|    12444|
|sensor-pad-16942T...|    16942|
|sensor-pad-180044...|    18004|
|device-mac-18129q...|    18129|
|sensor-pad-18532R...|    18532|
|device-mac-217770...|    21777|
|meter-gauge-26633...|    26633|
|therm-stick-27155...|    27155|
|device-mac-298713...|    29871|
|meter-gauge-31123...|    31123|
|meter-gauge-33119...|    33119|
|device-mac-37143D...|    37143|
|meter-gauge-40673...|    40673|
+--------------------+---------+
only showing top 20 rows

